In [9]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

sns.set(style="darkgrid")

In [10]:
df_odi = pd.read_csv('ODI-2022.csv',delimiter=";")

In [11]:
display(df_odi)

,Tijdstempel,What programme are you in?,Have you taken a course on machine learning?,Have you taken a course on information retrieval?,Have you taken a course on statistics?,Have you taken a course on databases?,What is your gender?,Chocolate makes you.....,When is your birthday (date)?,Number of neighbors sitting around you?,Did you stand up?,What is your stress level (0-100)?,"You can get 100 euros if you win a local DM competition, or we don’t hold any competitions and I give everyone some money (not the same amount!). How much do you think you would deserve then?",Give a random number,Time you went to be Yesterday,What makes a good day for you (1)?,What makes a good day for you (2)?
0,4-5-2022 12:12:07,artificial intelligence,yes,0,mu,nee,female,neither,02.05.1998,0,no,0,5,888,23:00,coffee,sun
1,4-5-2022 12:12:07,big data,yes,0,mu,nee,male,fat,29-02-2000,0,yes,200,1,12,32,7777,8964
2,4-5-2022 12:12:09,Computational Science,yes,1,mu,ja,male,fat,29-06-1998,1,no,0,100,9,x,0,0
3,4-5-2022 12:12:09,Bioinformatics and Systems Biology,yes,0,unknown,nee,male,neither,11/05/97,0,no,20,100,31,14:00,good food,no stress
4,4-5-2022 12:12:15,AI,yes,1,mu,ja,male,fat,03-10-1997,0,no,2,100,86,07:20,Sunshine,Good Food
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299,4-5-2022 12:35:31,Ai,yes,1,unknown,nee,not willing to answer,fat,1/1/1956,2,no,30,99,6,5,Yes,No
300,4-5-2022 12:37:05,E,yes,1,mu,ja,male,slim,9,9,yes,9,5,0,9,9,9
301,4-5-2022 12:41:59,MSc Computational Science,no,0,mu,nee,male,I have no idea what you are talking about,17/07/1995,0,no,666,1,123,555,Good food,Sleep more than 6 hours
302,4-5-2022 12:54:39,Artificial Intelligence,yes,0,mu,nee,not willing to answer,neither,17,0,yes,70,87,7,0,nice weather,good mood
